##Installing Necessary Packages

In [ ]:
# Install the necessary packages if they aren't already
!pip install pdfplumber openpyxl mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 88.2 MB/s eta 0:00:00


In [ ]:
!!pip install pdfplumber openpyxl mysql-connector-python



['Collecting pdfplumber',
 '  Downloading pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/42.8 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b42.8/42.8 kB\x1b \x1b2.2 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: openpyxl in /usr/local/lib/python3.11/dist-packages (3.1.5)',
 'Collecting mysql-connector-python',
 '  Downloading mysql_connector_python-9.3.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (7.2 kB)',
 'Collecting pdfminer.six==20250327 (from pdfplumber)',
 '  Downloading pdfminer_six-20250327-py3-none-any.whl.metadata (4.1 kB)',
 'Requirement already satisfied: Pillow>=9.1 in /usr/local/lib/python3.11/dist-packages (from pdfplumber) (11.2.1)',
 'Collecting pypdfium2>=4.18.0 (from pdfplumber)',
 '  Downloading pypdfium2-4.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (48 kB)',
 '\x1b[?25l 

##Connecting to Email Inbox

In [ ]:
import imaplib
import email
from email.header import decode_header
import os
import getpass
imap_server  = "imap.gmail.com"
email_user = input("Enter your email:")
password = getpass.getpass("Enter your password(use password for gmail):")
mail = imaplib.IMAP4_SSL(imap_server)
mail.login(email_user,password)
mail.select("inbox")

Enter your email:vigneshwarsah2153@gmail.com
Enter your password(use password for gmail):··········


('OK', [b'1060'])

##Fetching Email and Extracting Attachments

In [ ]:
def fetch_emails(mail,limits = 5):
  status,message = mail.search(None,"ALL")
  email_ids = message[0].split()[-limits:]
  emails = []
  for eid in email_ids:
    _msg_data = mail.fetch(eid,"(RFC822)")
    for response_part in _msg_data:
      if isinstance(response_part,tuple):
        msg = email.message_from_bytes(response_part[1])
        emails.append(msg)
  return emails

def parse_email(msg,download_folder = "/content/attachments"):
  subject = decode_header(msg["Subject"])[0][0]
  if isinstance(subject,bytes): subject = subject.decode(errors="ignore")
  _from = msg.get("From")
  body = " "
  attachments = []
  os.makedirs(download_folder,exist_ok=True)

  if msg.is_multipart():
    for part in msg.walk():
      content_type = part.get_content_type()
      content_disposition = str(part.get("Content-Disposition"))
      filename = part.get_filename()
      if filename:
        filepath = os.path.join(download_folder,filename)
        with open(filepath,"wb") as f:
          f.write(part.get_payload(decode=True))
        attachments.append(filepath)
      elif content_type == "text/plain":
        try:
          body += part.get_payload(decode=True).decode()
        except:

          continue
  else:
    body  = msg.get_payload(decode = True).decode(errors = "ignore")
  return subject,_from,body.strip(),attachments


##NLP Classification of Emails

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB


X_train = [
    "Please find the invoice attached",
    "Let's have a meeting tomorrow",
    "Special promotion on our product",
]
y_train = ["Invoice", "Meeting", "Marketing"]

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

def classify_email(text):
    X_input = vectorizer.transform([text])
    return clf.predict(X_input)[0]


##Extracting Structured Data from Attachments

In [ ]:
import pdfplumber
import pandas as pd
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
      return "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
def extract_data_from_excel(excel_path):
    return pd.read_excel(excel_path)

##Storing Data in Mysql Database

In [ ]:
import mysql.connector


db_config = {
    'host': '330',
    'user': 'root',
    'password': 'Saradha81#',
    'database': 'vignesh'
}

def store_email_data(subject, sender, body, category):
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS emails (
            id INT AUTO_INCREMENT PRIMARY KEY,
            subject TEXT,
            sender TEXT,
            body TEXT,
            category VARCHAR(50)
        )
    """)

    sql = "INSERT INTO emails (subject, sender, body, category) VALUES (%s, %s, %s, %s)"
    cursor.execute(sql, (subject, sender, body, category))
    conn.commit()
    cursor.close()
    conn.close()


##Running Full Pipeline

In [ ]:

def fetch_emails(mail,limits = 5):

  try:
      mail.noop()
  except imaplib.abort:

      print("Connection to IMAP server lost. Please re-run the connection setup.")
      raise

  status,message = mail.search(None,"ALL")
  email_ids = message[0].split()[-limits:]
  emails = []
  for eid in email_ids:

    try:
        _msg_data = mail.fetch(eid,"(RFC822)")
        for response_part in _msg_data:
          if isinstance(response_part,tuple):
            msg = email.message_from_bytes(response_part[1])
            emails.append(msg)
    except imaplib.abort:
        print(f"Failed to fetch email ID {eid}. Skipping.")
        continue

  return emails

def parse_email(msg,download_folder = "/content/attachments"):
  subject = decode_header(msg["Subject"])[0][0]
  if isinstance(subject,bytes): subject = subject.decode(errors="ignore")
  _from = msg.get("From")
  body = " "
  attachments = []
  os.makedirs(download_folder,exist_ok=True)

  if msg.is_multipart():
    for part in msg.walk():
      content_type = part.get_content_type()
      content_disposition = str(part.get("Content-Disposition"))
      filename = part.get_filename()
      if filename:
        filepath = os.path.join(download_folder,filename)
        with open(filepath,"wb") as f:
          f.write(part.get_payload(decode=True))
        attachments.append(filepath)
      elif content_type == "text/plain":
        try:
          body += part.get_payload(decode=True).decode()
        except:

          continue
  else:
    body  = msg.get_payload(decode = True).decode(errors = "ignore")
  return subject,_from,body.strip(),attachments

##Saving Attachments to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
